In [2]:
import argparse
import sys
import os
import traceback
import random
import ast
from tqdm import tqdm
tqdm.pandas()
import pandas as pd
from data_processing import DataFilter
from base_distillation import BaseDistillation  # 引用共享模块
sys.path.append('../')
from tool_llm_response.call_llm_with_zny import CallLLMByZny, ZnyConfig
from tool_kg_search.get_api_obs import shuffle_obs
sys.path.append('../')
from common import utils,utils_log


In [4]:
folder = '/mnt/pfs-guan-ssai/nlu/wangxiaoyuan/online-data/2024-09-24/prod/'
df = pd.read_csv(folder+'2024-09-24_rule_labeled.csv')
df

,msg-id,record_id,account_id,user-query,domain,tts,taskformer-model-13b-input,taskformer-model-13b-output,knowledge_search_result,content,...,query_length,assistant_has_chinese,assistant_has_repeat,assistant_chinese_ratio,assistant_length,question_has_repeat,context_llm_query_ratio,most_similar_train_query,similarity_to_train_query,is_too_similar_to_train
0,probe-msgId-9de0d65c521e4235b41d62cec48edf78,probe-msgId-9de0d65c521e4235b41d62cec48edf78,monitor-system-robot,今天的国际新闻,gpt_chat,以下是今天的国际新闻：\n1. 江西新余一企业突发火灾：9月23日19时许，新余市高新区一家...,NaN,NaN,NaN,NaN,...,-1,-1,-1,-1.000000,-1,-1,-1.0,NaN,NaN,-1
1,probe-msgId-daa4c3fd455447989b01a47cb5aca6a8,probe-msgId-daa4c3fd455447989b01a47cb5aca6a8,monitor-system-robot,平潭哪个海滩最值得去？,gpt_chat,平潭有许多值得去的海滩，以下几个海滩各具特色，推荐你去：\n1. **北澳月海湾**：这是平...,NaN,NaN,NaN,NaN,...,-1,-1,-1,-1.000000,-1,-1,-1.0,NaN,NaN,-1
2,probe-msgId-d3b2ee90a0364cf2b5f6017b29c18ab1,probe-msgId-d3b2ee90a0364cf2b5f6017b29c18ab1,monitor-system-robot,明天天气怎么样,weather,暂时无法获取位置信息。,NaN,NaN,NaN,NaN,...,-1,-1,-1,-1.000000,-1,-1,-1.0,NaN,NaN,-1
3,probe-msgId-2462a41269e4490f889585dc8f40193e,probe-msgId-2462a41269e4490f889585dc8f40193e,monitor-system-robot,中国茅台这个股票走势怎么样,stock,贵州茅台当前价格1261.54，跌幅0.19%,NaN,NaN,NaN,NaN,...,-1,-1,-1,-1.000000,-1,-1,-1.0,NaN,NaN,-1
4,probe-msgId-61d482f8e3624fd295c7373c556e0192,probe-msgId-61d482f8e3624fd295c7373c556e0192,monitor-system-robot,环球影城好玩吗,gpt_chat,环球影城非常好玩，尤其是北京环球影城，包含了多个主题景区和丰富多样的娱乐项目。以下是一些主要...,NaN,NaN,NaN,NaN,...,-1,-1,-1,-1.000000,-1,-1,-1.0,NaN,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,0B5A2C8F-F3EB-49DF-8B68-0A5678CCAE95,0B5A2C8F-F3EB-49DF-8B68-0A5678CCAE95,3453957508393304066,十一假期后面下一个假期是啥时候,calendar,距离国庆节还有7天。,NaN,NaN,NaN,NaN,...,-1,-1,-1,-1.000000,-1,-1,-1.0,NaN,NaN,-1
458,42e4decb-63f4-4011-a586-37630b720aef,69524c9d-7fb7-446c-b2cc-c8a86ba24836,3682641316864122881,大学学习委员的总体目标是什么,gpt_chat,大学学习委员的总体目标主要包括以下几个方面：\n1. **积极营造良好的学风**，促进班级学...,[unused0]system\n你是一个名字叫做理想同学的AI数字生命体。\n理想同学是一...,大学学习委员的总体目标主要包括以下几个方面：\n1. **积极营造良好的学风**，促进班级学...,"[{""api_type"":""QASearch"",""thought"":""涉及知识查询，查询大学...",NaN,...,14,1,0,0.765823,166,0,-1.0,昆山工研院的主要目标和愿景是什么？,0.79,0
459,6040DF35-AF67-4CD6-8B42-376CC1FD6B1A,6040DF35-AF67-4CD6-8B42-376CC1FD6B1A,3453957508393304066,十一假期后面的下一个假期是啥时候,calendar,距离国庆节还有7天。,NaN,NaN,NaN,NaN,...,-1,-1,-1,-1.000000,-1,-1,-1.0,NaN,NaN,-1
460,8DA826E8-1FEE-404D-AA96-A09DFEDEE9C1,8DA826E8-1FEE-404D-AA96-A09DFEDEE9C1,3399679939340054529,招商银行今天的股价,stock,招商银行当前价格33.13，涨幅5.54%,NaN,NaN,NaN,NaN,...,-1,-1,-1,-1.000000,-1,-1,-1.0,NaN,NaN,-1
